<a href="https://colab.research.google.com/github/Hemanth0411/Machine-Learning/blob/main/Project_Fake_News_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing the dataset

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c fake-news

 99% 46.0M/46.5M [00:00<00:00, 93.9MB/s]
100% 46.5M/46.5M [00:00<00:00, 83.6MB/s]


In [ ]:
!unzip fake-news.zip

Archive:  fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


In [20]:
import pandas as pd
data = pd.read_csv("train.csv")
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [21]:
data = data.dropna()
data.reset_index(inplace=True)

In [22]:
x=data.drop("label", axis=1)
y=data["label"]

In [23]:
print(x.shape)
print(y.shape)

(18285, 5)
(18285,)


## Importing required libraries

In [24]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

## Preprocessing

In [25]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
messages = x.copy()
messages.head()
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
  review = re.sub("[^a-zA-Z]", " ", messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words("english")]
  review = " ".join(review)
  corpus.append(review)

In [28]:
# One hot encoding

vocab_size = 5000
onehot_repr = [one_hot(words, vocab_size) for words in corpus]
onehot_repr

[[1981, 2141, 2984, 4153, 241, 387, 2966, 624, 4914, 2345],
 [3730, 1482, 697, 197, 243, 2885, 4230],
 [1662, 3946, 1756, 2798],
 [2350, 756, 1947, 3662, 2201, 4343],
 [3008, 243, 1703, 803, 4279, 4598, 243, 199, 2427, 326],
 [4910,
  4151,
  4213,
  3174,
  4943,
  4414,
  3988,
  3718,
  1490,
  781,
  2479,
  3414,
  1770,
  1683,
  4230],
 [1112, 1111, 4270, 2944, 3295, 1325, 3961, 323, 43, 289, 3920],
 [2234, 1823, 1999, 3280, 1864, 4009, 4414, 1953, 43, 289, 3920],
 [3305, 4635, 2382, 2446, 3407, 1845, 680, 1749, 4414, 3332],
 [4282, 2361, 2501, 2417, 4674, 365, 1565, 3335],
 [4477, 4084, 4042, 2508, 4816, 1271, 1938, 3582, 3385, 4835, 4812],
 [3662, 1847, 241, 1845, 4414, 1864],
 [4776, 267, 2574, 1707, 1976, 328, 166, 3997, 583],
 [410, 3222, 663, 2470, 808, 2444, 4503, 43, 289, 3920],
 [3714, 3298, 3225, 1175, 4072, 43, 289, 3920],
 [625, 418, 4449, 1538, 4405, 2738, 3877, 2288, 1657, 4110],
 [2905, 1036, 1482],
 [546, 3621, 2130, 3248, 4414, 145, 727, 4230],
 [3510, 3341, 697

In [29]:
# Applying pad sequences to make sentences length equal
sent_len = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_len)
embedded_docs

array([[   0,    0,    0, ...,  624, 4914, 2345],
       [   0,    0,    0, ...,  243, 2885, 4230],
       [   0,    0,    0, ..., 3946, 1756, 2798],
       ...,
       [   0,    0,    0, ...,   43,  289, 3920],
       [   0,    0,    0, ...,  509, 3168,  355],
       [   0,    0,    0, ..., 2034, 4572, 1755]], dtype=int32)

# Creating Model

In [31]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_features))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [32]:
import numpy as np
x_final = np.array(embedded_docs)
y_final = np.array(y)

In [33]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42)

In [34]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.7953 - loss: 0.4191 - val_accuracy: 0.9173 - val_loss: 0.1937
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - accuracy: 0.9457 - loss: 0.1422 - val_accuracy: 0.9213 - val_loss: 0.2202
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.9670 - loss: 0.0962 - val_accuracy: 0.9185 - val_loss: 0.2254
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9787 - loss: 0.0686 - val_accuracy: 0.9138 - val_loss: 0.2343
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - accuracy: 0.9838 - loss: 0.0474 - val_accuracy: 0.9143 - val_loss: 0.2799
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 42ms/step - accuracy: 0.9908 - loss: 0.0354 - val_accuracy: 0.9157 - val_loss: 0.3675
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 9s 34ms/step - accuracy: 0.9957 - loss: 0.0168 - val_accuracy: 0.9160 - val_loss: 0.4581
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 11s 56ms/step - accuracy: 0.9958 - loss: 0.0141 - val_

## Performance metrics

In [35]:
y_pred = model.predict(x_test)
y_pred = np.round(y_pred).astype(int)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)


189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


array([[3095,  324],
       [ 207, 2409]])

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.912013256006628